# intermediate_IMDB
This notebook takes our IMDB dataset and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [1]:
import os
os.chdir('..')
# os.getcwd( )

In [2]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [4]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_IMDB_01"

# Ensure we're on an ARM environment if necessary.
platform_check()

We're Armed: macOS-13.1-arm64-i386-64bit


## Load Data

### IMDB

In [4]:
dataset_path = 'data/inter_IMDB_sentiment/IMDB_preped_train.csv'
df = pd.read_csv(dataset_path)

df.head()

,text,label,num_word_text
0,This film has been compared to the hilarious B...,0,186
1,Reasonably effective horror/science-fiction a ...,1,61
2,"The inspiration for the ""Naked Gun"" movies cas...",1,169
3,When this film was originally released it was ...,1,634
4,I happened upon this by chance. I was at my fr...,1,334


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   text           40000 non-null  object
 1   label          40000 non-null  int64 
 2   num_word_text  40000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 937.6+ KB


In [6]:
df['label'].value_counts()

1    20044
0    19956
Name: label, dtype: int64

### Target Text & Labels

In [7]:
text = df.text.values
labels = df.label.values

## Preprocess

In [8]:
token_id = []
attention_masks = []

for sample in text:
  encoding_dict = preprocessing(sample, params.tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [9]:
val_ratio = 0.2

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels,
    random_state=1)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = RandomSampler(val_set),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
        )

In [10]:
one = 0
zero = 0
for i in train_set:
    if i[2] == 1:
        one+=1
    elif i[2] == 0:
        zero += 1
         
print("one", one)
print("zero", zero)

one 16035
zero 15965


## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [10]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Layer (type:depth-idx)                                       Output Shape              Param #
RobertaForSequenceClassification                             [1, 2]                    --
├─RobertaModel: 1-1                                          [1, 512, 768]             --
│    └─RobertaEmbeddings: 2-1                                [1, 512, 768]             --
│    │    └─Embedding: 3-1                                   [1, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [1, 512, 768]             768
│    │    └─Embedding: 3-3                                   [1, 512, 768]             394,752
│    │    └─LayerNorm: 3-4                                   [1, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [1, 512, 768]             --
│    └─RobertaEncoder: 2-2                                   [1, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --               

Set model to device, initialize trainer

In [11]:
model.to(params.device)
print(f"Trained Dataset: {dataset_path}")
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

Trained Dataset: data/inter_IMDB_sentiment/IMDB_preped_train.csv
Device: mps


Fit the model to our training data.

In [12]:
trainer.fit()

Epoch 1:   0%|          | 0/2000 [00:00<?, ?batch/s]/Users/jarradjinx/opt/anaconda3/envs/itesd_env/lib/python3.9/site-packages/transformers/models/roberta/modeling_roberta.py:1570: UserWarning: The operator 'aten::cumsum.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  incremental_indices = (torch.cumsum(mask, dim=1).type_as(mask) + past_key_values_length) * mask
	 Validation 499: 100%|██████████| 500/500 [03:18<00:00,  2.53batch/s]


	 - Train loss: 0.213777
	 - Validation Loss: 0.163193
	 - Validation Accuracy: 0.938000
	 - Validation F1: 0.934431
	 - Validation Recall: 0.943260
	 - Validation Precision: 0.934511
	 * Model @ epoch 1 saved to model_saves/intermediate_IMDB_01/E01_A0.94_F0.93


	 Validation 499: 100%|██████████| 500/500 [03:18<00:00,  2.52batch/s]


	 - Train loss: 0.134534
	 - Validation Loss: 0.160835
	 - Validation Accuracy: 0.940500
	 - Validation F1: 0.937554
	 - Validation Recall: 0.957213
	 - Validation Precision: 0.927027
	 * Model @ epoch 2 saved to model_saves/intermediate_IMDB_01/E02_A0.94_F0.94


	 Validation 499: 100%|██████████| 500/500 [03:18<00:00,  2.52batch/s]


...

	 - Train loss: 0.020455
	 - Validation Loss: 0.283110
	 - Validation Accuracy: 0.938500
	 - Validation F1: 0.935669
	 - Validation Recall: 0.951117
	 - Validation Precision: 0.928768
	 * Model @ epoch 8 saved to model_saves/intermediate_IMDB_01/E08_A0.94_F0.94


	 Validation 499: 100%|██████████| 500/500 [03:18<00:00,  2.52batch/s]


	 - Train loss: 0.018911
	 - Validation Loss: 0.366701
	 - Validation Accuracy: 0.934500
	 - Validation F1: 0.932137
	 - Validation Recall: 0.959832
	 - Validation Precision: 0.914201
	 * Model @ epoch 9 saved to model_saves/intermediate_IMDB_01/E09_A0.93_F0.93


	 Validation 499: 100%|██████████| 500/500 [03:18<00:00,  2.52batch/s]


	 - Train loss: 0.015637
	 - Validation Loss: 0.365525
	 - Validation Accuracy: 0.934125
	 - Validation F1: 0.932048
	 - Validation Recall: 0.955656
	 - Validation Precision: 0.918202
	 * Model @ epoch 10 saved to model_saves/intermediate_IMDB_01/E10_A0.93_F0.93
